# 요약
** 결측치가 절대 0이면 안되는 이유

input data의 하루 동안의 변화량(적분값)-> target data rate


eg)

    1. 광량↑ 잎면적 ↑↑
    2. 광량 - 잎면적 ↑/-
    3. 광량↓ 잎면적 -/↓
    4. 광량 0 잎면적 ↓

    하루 동안의 광량 -> 잎면적 증감 관련
    ie) '하루 동안의 데이터 그래프 적분값'이 곧 변화량
        -> [day n]의 그래프 적분값 - [day n-1]의 그래프 적분값
        => [day n]의 target rate와 비례관계


* 하려고 했던 것
1. 한 컬럼을 골라 하루치에 해당하는 데이터를 그래프로 그려보고 적분값을 구해본 뒤
2. (1번이 할만하다 싶으면) 한 CASE를 골라 전부 적분값을 구해보고
3. 구한 적분값을 바탕으로 연산해서(x값), target rate 누적곱(y값) 그래프 그리려고 했음
* 코딩&서치 능력 부족으로 시간 내 달성 실패함..
* 그리고 구하기 쉽게 csv 파일을 하나로 합치려고 했는데 중간 부분만 합쳐지는 이유를 모르겠음.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/DACON_PakChoi

/content/drive/MyDrive/Colab Notebooks/DACON_PakChoi


In [3]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn # NN Base code(Class)
import torch.optim as optim # Optimizer
import torch.nn.functional as F # NN Base code(Function)
from torch.utils.data import Dataset, DataLoader # Custom Dataset

from tqdm.auto import tqdm # Progress bar

import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
# TO USE GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Fixed RandomSeed

# Data Pre-processing

In [30]:
'''
file_format = ".csv"
file_path = "./train_input/"
file_list = glob.glob(f"{file_path}/*{file_format}")

df = pd.DataFrame()
for file_name in file_list:
    file_df = pd.read_csv(file_name, encoding='utf-8') 

columns = list(file_df.columns)
temp_df = pd.DataFrame(file_df, columns=columns)
df = df.append(temp_df, ignore_index=False)

df.to_csv("train_merged.csv", index=False)
'''

'\nfile_format = ".csv"\nfile_path = "./train_input/"\nfile_list = glob.glob(f"{file_path}/*{file_format}")\n\ndf = pd.DataFrame()\nfor file_name in file_list:\n    file_df = pd.read_csv(file_name, encoding=\'utf-8\') \n\ncolumns = list(file_df.columns)\ntemp_df = pd.DataFrame(file_df, columns=columns)\ndf = df.append(temp_df, ignore_index=False)\n\ndf.to_csv("merged.csv", index=False)\n'

In [38]:
'''
file_format = ".csv"
file_path = "./train_target/"
file_list = glob.glob(f"{file_path}/*{file_format}")

df = pd.DataFrame()
for file_name in file_list:
    file_df = pd.read_csv(file_name, encoding='utf-8') 

columns = list(file_df.columns)
temp_df = pd.DataFrame(file_df, columns=columns)
df = df.append(temp_df, ignore_index=False)

df.to_csv("target_merged.csv", index=False)
'''

'\nfile_format = ".csv"\nfile_path = "./train_target/"\nfile_list = glob.glob(f"{file_path}/*{file_format}")\n\ndf = pd.DataFrame()\nfor file_name in file_list:\n    file_df = pd.read_csv(file_name, encoding=\'utf-8\') \n\ncolumns = list(file_df.columns)\ntemp_df = pd.DataFrame(file_df, columns=columns)\ndf = df.append(temp_df, ignore_index=False)\n\ndf.to_csv("target_merged.csv", index=False)\n'

In [37]:
df_input = pd.read_csv('input_merged.csv')
df_target = pd.read_csv('target_merged.csv')

In [43]:
print(df_input.shape)
print(df_target.shape)

(46080, 38)
(33, 2)


In [44]:
df_input1 = pd.read_csv('./train_input/CASE_01.csv') # 2021-02-17 ~ 2021-03-17
df_target1 = pd.read_csv('./train_target/CASE_01.csv') # 2021-02-18 ~ 2021-03-18

In [ ]:
'''
내부온도관측치
내부습도관측치
CO2관측치
EC관측치
외부온도관측치
외부습도관측치
펌프상태
펌프작동남은시간
최근분무량
일간누적분무량
냉방상태
냉방작동남은시간
난방상태
난방작동남은시간
내부유동팬상태
내부유동팬작동남은시간
외부환기팬상태
외부환기팬작동남은시간
화이트 LED상태
화이트 LED작동남은시간
화이트 LED동작강도
레드 LED상태
레드 LED작동남은시간
레드 LED동작강도
블루 LED상태
블루 LED작동남은시간
블루 LED동작강도
카메라상태
냉방온도
난방온도
기준온도
난방부하
냉방부하
총추정광량
백색광추정광량
적색광추정광량
청색광추정광량
'''

In [45]:
white_df = df_input1['백색광추정광량']


In [51]:
for nn in range(len(white_df)):
  if white_df.loc[nn] != 0.00:
    print(white_df[nn], df_input1['시간'].iloc[nn])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
309.41 2021-03-12 09:31:00
309.41 2021-03-12 09:32:00
309.41 2021-03-12 09:33:00
309.41 2021-03-12 09:34:00
309.41 2021-03-12 09:35:00
309.41 2021-03-12 09:36:00
309.41 2021-03-12 09:37:00
309.41 2021-03-12 09:38:00
309.41 2021-03-12 09:39:00
309.41 2021-03-12 09:40:00
309.41 2021-03-12 09:41:00
309.41 2021-03-12 09:42:00
309.41 2021-03-12 09:43:00
309.41 2021-03-12 09:44:00
309.41 2021-03-12 09:45:00
309.41 2021-03-12 09:46:00
309.41 2021-03-12 09:47:00
309.41 2021-03-12 09:48:00
309.41 2021-03-12 09:49:00
309.41 2021-03-12 09:50:00
309.41 2021-03-12 09:51:00
309.41 2021-03-12 09:52:00
309.41 2021-03-12 09:53:00
309.41 2021-03-12 09:54:00
309.41 2021-03-12 09:55:00
309.41 2021-03-12 09:56:00
309.41 2021-03-12 09:57:00
309.41 2021-03-12 09:58:00
309.41 2021-03-12 09:59:00
309.41 2021-03-12 10:00:00
309.41 2021-03-12 10:01:00
309.41 2021-03-12 10:02:00
309.41 2021-03-12 10:03:00
309.41 2021-03-12 10:04:00
309.41 2021-03-12 10:05:00
309.41 2021-03-12 1